In [0]:
order_file_path='/Volumes/e_comm_data_modeling/default/bronze_e_comm_data/bronze_raw_order_items.csv'
selling_file_path='/Volumes/e_comm_data_modeling/default/bronze_e_comm_data/bronze_sellers.csv'

In [0]:
raw_order_df=spark.read.option('header',True)\
        .option('inferschema',True).csv(order_file_path) 
raw_seller_df=spark.read.option('header',True)\
    .option('inferschema',True).csv(selling_file_path) 

In [0]:
raw_seller_df.printSchema()
raw_order_df.printSchema()

In [0]:
# Simple validations for bronze layer
from pyspark.sql import functions as F
raw_order_df=raw_order_df.dropna(how="all")\
    .filter(F.col('order_id').isNotNull() & F.col('customer_id').isNotNull() & F.col('seller_id').isNotNull() & F.col('address_id').isNotNull())

raw_seller_df=raw_seller_df.dropna(how="all")\
    .filter(F.col('seller_id').isNotNull())

In [0]:
# raw_seller_df.printSchema()
# raw_order_df.printSchema()

In [0]:
%sql
-- Will craete a delta table which will hold daily order 
-- here we are going to append the data daily by partitioning the data by order_date

CREATE TABLE IF NOT EXISTS e_comm_data_modeling.default.raw_order (
  order_id STRING,
  product_id STRING,
  product_name STRING,
  category STRING,
  seller_id STRING,
  customer_id STRING,
  customer_name STRING,
  address_id STRING,
  order_date DATE,
  delivery_date DATE,
  quantity INT,
  unit_price INT,
  discount INT,
  is_cancelled INT,
  is_returned INT
) 
USING DELTA
TBLPROPERTIES (quality = 'bronze')

In [0]:
raw_order_df.write.format('delta').mode('append').insertInto('e_comm_data_modeling.default.raw_order')
raw_seller_df.write.format('delta').mode('overwrite').saveAsTable('e_comm_data_modeling.default.seller_data')

#.save()  -- it will only store the data will not keep any metadata about the file
#.saveasTable --it  will create the table and store into it and will maintain the metadata in metastore
#.insertInto() -- if u created the table already and u want to insert the data into the table go for it.